## BEMM458 Final Assignment

## You are a data scientist working for the European Union. There is an increasing concern inn  relation to the spread and amplification of misinformation on social media.

## General management needs you to conduct an analysis of recent conversations posted on Twitter in relation to spain. You have to identify ongoing misinformation campaigns and profile those users contributing to the spread of misinformation.




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scipy
import re

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
pd.set_option('display.max_rows', 120)

In [4]:
pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
spainEnglishAll=pd.read_csv('./BEMM458FinalAssignment.csv')

In [6]:
spainEnglishAll.columns

Index(['Unnamed: 0', 'tweeet.id', 'tweet.text', 'tweet.created_at',
       'tweet.author_id', 'tweet.lang', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count'],
      dtype='object')

In [7]:
spainEnglishAll

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421,en,1568,0,0
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653,en,38,0,0
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080,en,90,0,0
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768,en,0,0,5
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224,en,34,0,0
...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504,en,0,0,0
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413,en,18,0,0
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352,en,0,0,1
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160,en,121,0,0


## Question 1. Which twitter conversations have large levels of amplification ? (15 points)
### Tip: consider number of retweets, likes, etc as proxies to measure the spread of information on social media

### <font color='green'> Answer (First Part): First, we need to find approximate  amplification relation for the retweets, reply and like counts. If our goal is the finding amplification of misinformation, we need to look which terms reach the tweet to the people's screen most. Retweet of tweet directly shows the tweet for all the followers of that account. Reply number should be less attribute to spreading misinformation, because the person who replied can also state that tweet has misinformation. However, reply count still very important at spreading of misinformation because replying moves that tweet to the person's followers' main screen. For like count, we can see some liked tweets of accounts that we are following in our twitter main screen, but frequency of it so low when it compared with other 2 parameters. However, like count has a meaning of confirmation on twitter and it can affect people's acceptance for that misinformation, so it has still some effect for amplification of misinformation. To conclude, according to their effects, we need assign coefficients to these terms and multiply with it. Then, we can sum up these counts and create new column as amplification_volume. There is no concrete coefficients for these terms, but even if we give bigger coefficient number according to their power of amplification, it would be more meaningful analyst rather than just count numbers.
Coefficient of Retweet : 5 --- Coefficient of Reply : 4 --- Coefficient of Like = 2 </font>

In [8]:
spainEnglishAll["amplification_volume"] = (spainEnglishAll["public_metrics.retweet_count"]*5) + (spainEnglishAll["public_metrics.like_count"] * 2) + (spainEnglishAll["public_metrics.reply_count"]*4)

In [9]:
spainEnglishAll

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421,en,1568,0,0,7840
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653,en,38,0,0,190
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080,en,90,0,0,450
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768,en,0,0,5,10
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224,en,34,0,0,170
...,...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504,en,0,0,0,0
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413,en,18,0,0,90
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352,en,0,0,1,2
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160,en,121,0,0,605


In [10]:
amp_sorted = spainEnglishAll.sort_values("amplification_volume", ascending=False)

In [11]:
amp_sorted.head(100)

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
23467,23467,1461162679809155076,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 02:41:44+00:00,1290910527393210368,en,44845,0,0,224225
23656,23656,1461157780664901633,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 02:22:16+00:00,1101831422631804800,en,43945,0,0,219725
8820,8820,1461371211343020033,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 16:30:22+00:00,1168957082,en,43945,0,0,219725
39357,39357,1461649200882405377,RT @NCTsmtown_127: Spain here we goooooo\n\n#N...,2021-11-19 10:55:00+00:00,1064006291087970432,en,35930,0,0,179650
45373,45373,1461537918900600843,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-19 03:32:48+00:00,1382234880465989632,en,34538,0,0,172690
41908,41908,1461613145219239944,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-19 08:31:43+00:00,1388490486181076992,en,34538,0,0,172690
23655,23655,1461157789397438464,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 02:22:18+00:00,1101831422631804800,en,34537,0,0,172685
45333,45333,1461539133520695297,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-19 03:37:38+00:00,1382234880465989632,en,32878,0,0,164390
45374,45374,1461537902106537991,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-19 03:32:44+00:00,1382234880465989632,en,32632,0,0,163160
44456,44456,1461565440879206401,RT @NCTsmtown_127: #Johnny 's Photo Diary:Spai...,2021-11-19 05:22:10+00:00,1322715465198821376,en,30490,0,0,152450


### <font color='green'>Answer(last part): Tweets shown above have largest level of amplification volume of this dataset. Tweets that have same amplification_volume are same retweeted tweets by different accounts.</font>

## Question 2. Analysis of highly amplified conversations (15 points)

### Please describe  5 highly amplified conversations which are trying to polarize or misinform.

## Tip: given a tweet.id it is possible to have a detailed look at twitter by appending this id to the url: https://twitter.com/anyuser/status/


### For instance given the tweet.id: 1461577564850319362 you can inspect its content by having a look at: https://twitter.com/anyuser/status/1461577564850319362



In [12]:
grouped_tweet_text = spainEnglishAll.groupby(["tweet.text"]).mean("amplification_volume").sort_values("amplification_volume", ascending=False)

In [13]:
grouped_tweet_text.shape

(22785, 6)

In [14]:
22785*2 #means that, there are some tweets got retweet from different accounts in these dataset.

45570

###  <font color='green'> This result means that, there are some tweets got retweet from different accounts in this dataset.</font>

In [15]:
grouped_tweet_text.head(25)

,Unnamed: 0,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
tweet.text,,,,,,
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography \n#NCT #NCT127 #JAEHYUN\n#Spain #Sevilla https://t.co/pzRbU9G82o,23467,1290910527393210368,44845,0,0,224225
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography\n#NCT #NCT127\n#JAEHYUN #MARK \n#Spain #Sevilla https://t.co/jPNpBlVUcJ,16238,550915711900380928,43945,0,0,219725
RT @NCTsmtown_127: Spain here we goooooo\n\n#NCT #NCT127 #MARK\n#NCT127inSpain https://t.co/3kuRW5nG1f,39357,1064006291087970432,35930,0,0,179650
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography\n#NCT #NCT127 #DOYOUNG \n#Spain #Sevilla https://t.co/XFbjF6fXt2,36979,1290852263092957184,34538,0,0,172688
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#NCT #NCT127 \n#Spain #Sevilla https://t.co/2xGKANRyR2,45333,1382234880465989632,32878,0,0,164390
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography \n#NCT #NCT127\n#Spain #Sevilla https://t.co/11xjaXYrMh,45374,1382234880465989632,32632,0,0,163160
RT @NCTsmtown_127: #Johnny 's Photo Diary:Spain\n\n#Johntography \n#NCT #NCT127 #JAEHYUN\n#Spain #Sevilla https://t.co/7kFerMPKyq,33960,1306812996296015872,30490,0,0,152448
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n📸 by #DOYOUNG \n\n#NCT #NCT127\n#Spain #Sevilla https://t.co/0Vn0A4EPd4,45332,1382234880465989632,30038,0,0,150190
RT @NCTsmtown_127: #Johnny 's Photo Diary:Spain\n\n#NCT #NCT127\n#Spain #Sevilla https://t.co/UAqpnZa6Wt,45336,1382234880465989632,27322,0,0,136610


In [16]:
NCTsmtown_event = grouped_tweet_text[0:17]

In [17]:
NCTsmtown_event

,Unnamed: 0,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
tweet.text,,,,,,
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography \n#NCT #NCT127 #JAEHYUN\n#Spain #Sevilla https://t.co/pzRbU9G82o,23467,1290910527393210368,44845,0,0,224225
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography\n#NCT #NCT127\n#JAEHYUN #MARK \n#Spain #Sevilla https://t.co/jPNpBlVUcJ,16238,550915711900380928,43945,0,0,219725
RT @NCTsmtown_127: Spain here we goooooo\n\n#NCT #NCT127 #MARK\n#NCT127inSpain https://t.co/3kuRW5nG1f,39357,1064006291087970432,35930,0,0,179650
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography\n#NCT #NCT127 #DOYOUNG \n#Spain #Sevilla https://t.co/XFbjF6fXt2,36979,1290852263092957184,34538,0,0,172688
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#NCT #NCT127 \n#Spain #Sevilla https://t.co/2xGKANRyR2,45333,1382234880465989632,32878,0,0,164390
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography \n#NCT #NCT127\n#Spain #Sevilla https://t.co/11xjaXYrMh,45374,1382234880465989632,32632,0,0,163160
RT @NCTsmtown_127: #Johnny 's Photo Diary:Spain\n\n#Johntography \n#NCT #NCT127 #JAEHYUN\n#Spain #Sevilla https://t.co/7kFerMPKyq,33960,1306812996296015872,30490,0,0,152448
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n📸 by #DOYOUNG \n\n#NCT #NCT127\n#Spain #Sevilla https://t.co/0Vn0A4EPd4,45332,1382234880465989632,30038,0,0,150190
RT @NCTsmtown_127: #Johnny 's Photo Diary:Spain\n\n#NCT #NCT127\n#Spain #Sevilla https://t.co/UAqpnZa6Wt,45336,1382234880465989632,27322,0,0,136610


### <font color='green'> The biggest conversation that has large amplification of misinformation is NCT 127 pop group in given tweets or dates. Some accounts share old pictures of the group, in order to show them like they are in Spain. According to https://www.newsnow.co.uk/h/Entertainment+&+Arts/Music/K-Pop/SM+Entertainment/NCT, group was in USA at given dates. These conversations can be part of marketing operation by bots or fake accounts. Aim of operation can be increase the popularity of the group.</font>

In [18]:
spainEnglishAll.iloc[2252]

Unnamed: 0                                                                   2252
tweeet.id                                                     1461451876000489474
tweet.text                      RT @MesutOzil1088: I grew up as a @Fenerbahce ...
tweet.created_at                                        2021-11-18 21:50:54+00:00
tweet.author_id                                               1245113611263057920
tweet.lang                                                                     en
public_metrics.retweet_count                                                17801
public_metrics.reply_count                                                      0
public_metrics.like_count                                                       0
amplification_volume                                                        89005
Name: 2252, dtype: object

In [19]:
grouped_tweet_text[17:18]

,Unnamed: 0,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
tweet.text,,,,,,
RT @MesutOzil1088: I grew up as a @Fenerbahce fan as a kid in Germany - every German-Turkish person supports a Turkish team when they grow…,2252,1245113611263057920,17801,0,0,89005


###  <font color='green'>Second conversation that had big amplification volume is a tweet that can used by people who want to polarise the country. In these tweet, Mesut Ozil who is a famous football player, claim that Real Madrid is the biggest team of Spain and by saying that he created suitable opportunity to people who can increase the polarisation in Spain.</font>

In [20]:
spainEnglishAll.iloc[43940]

Unnamed: 0                                                                  43940
tweeet.id                                                     1461577564850319362
tweet.text                      RT @LAPXURADELBUIO: what is happening in spain...
tweet.created_at                                        2021-11-19 06:10:20+00:00
tweet.author_id                                               1324927883446345728
tweet.lang                                                                     en
public_metrics.retweet_count                                                16466
public_metrics.reply_count                                                      0
public_metrics.like_count                                                       0
amplification_volume                                                        82330
Name: 43940, dtype: object

In [21]:
grouped_tweet_text[18:19]

,Unnamed: 0,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
tweet.text,,,,,,
RT @LAPXURADELBUIO: what is happening in spain - a thread:,43940,1324927883446345728,16466,0,0,82330


### <font color='green'>Third conversation that had big amplification volume in these dataset is a retweet of a tweet that published in 5 Jul 2021. This tweet emphasise that Spanish policemen are targeting Lgbtq2+ community in Spain. This post is clearly misinformation about Spain.</font>

In [22]:
spainEnglishAll.iloc[34370]

Unnamed: 0                                                                  34370
tweeet.id                                                     1461716582602657792
tweet.text                      RT @AfricanArchives: African moors, who conque...
tweet.created_at                                        2021-11-19 15:22:45+00:00
tweet.author_id                                               1366298396051582976
tweet.lang                                                                     en
public_metrics.retweet_count                                                16115
public_metrics.reply_count                                                      0
public_metrics.like_count                                                       0
amplification_volume                                                        80575
Name: 34370, dtype: object

In [23]:
grouped_tweet_text[19:20]

,Unnamed: 0,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
tweet.text,,,,,,
"RT @AfricanArchives: African moors, who conquered Spain and ruled it for 700 years, were responsible for bringing Europe out of the dark ag…",34370,1366298396051582976,16115,0,0,80575


### <font color='green'>Fourth conversation that has big amplification volume is about history. In this conversation, some account claim that black African people introduce education to the European people. This claim is misinformation. Because people who invaded Spain was leading by Arabic dynasty. In addition to that, even Arabic state had further education quality against Europeans, their effects cannot be only reason of increasing education level in West.</font>

In [24]:
spainEnglishAll.iloc[41984]

Unnamed: 0                                                                  41984
tweeet.id                                                     1461611790907809795
tweet.text                      RT @TheeAzanian: Normalize posting these poor ...
tweet.created_at                                        2021-11-19 08:26:21+00:00
tweet.author_id                                               1412044637288779776
tweet.lang                                                                     en
public_metrics.retweet_count                                                13620
public_metrics.reply_count                                                      0
public_metrics.like_count                                                       0
amplification_volume                                                        68100
Name: 41984, dtype: object

In [25]:
grouped_tweet_text[20:21]

,Unnamed: 0,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
tweet.text,,,,,,
"RT @TheeAzanian: Normalize posting these poor parts of Europe like they do with Africa. Cañada Real (just outside of Madrid), Spain. https:…",24472,1332043128319223808,13620,0,0,68100


### <font color='green'>Fifth biggest conversation is a retweet of a tweet like previous conversations. This tweet has misinformation. Because people in the pictures claimed that they are from Spain and they are living in very poor conditions. However, people in the reply section say that they are refugees or this pictures have taken in some Asian country. There is no clear evidence that where is the pictures have taken and who the settlers are in the pictures. </font>

## Question 3. Compute the z-score of the column "public_metrics.retweet_count" (15 points)
## tip: refer to the definition of Z-score at" https://www.statisticshowto.com/probability-and-statistics/z-score/

In [26]:
spainEnglishAll

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421,en,1568,0,0,7840
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653,en,38,0,0,190
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080,en,90,0,0,450
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768,en,0,0,5,10
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224,en,34,0,0,170
...,...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504,en,0,0,0,0
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413,en,18,0,0,90
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352,en,0,0,1,2
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160,en,121,0,0,605


In [27]:
np_retweet_counts = np.array(spainEnglishAll["public_metrics.retweet_count"])

In [28]:
np.nanmean(np_retweet_counts)

233.72148011022992

In [29]:
np.nanstd(np_retweet_counts)

959.4851708357776

In [30]:
z_score = (np_retweet_counts-np.nanmean(np_retweet_counts)) / np.nanstd(np_retweet_counts)

In [31]:
z_score

array([ 1.39061922, -0.20398594, -0.1497902 , ..., -0.24359051,
       -0.11748121, -0.24359051])

In [32]:
z_score.shape

(47550,)

In [33]:
z_score[0]

1.3906192200214222

In [34]:
spainEnglishAll["z_score_retweet"] = (spainEnglishAll["public_metrics.retweet_count"]-spainEnglishAll["public_metrics.retweet_count"].mean()) / spainEnglishAll["public_metrics.retweet_count"].std()

In [35]:
spainEnglishAll

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421,en,1568,0,0,7840,1
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653,en,38,0,0,190,-0
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080,en,90,0,0,450,-0
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768,en,0,0,5,10,-0
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224,en,34,0,0,170,-0
...,...,...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504,en,0,0,0,0,-0
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413,en,18,0,0,90,-0
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352,en,0,0,1,2,-0
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160,en,121,0,0,605,-0


## Question 4. Filter conversations with a z-score larger than 10. (15 points)

In [36]:
filter_z_score = []

In [37]:
for i in z_score:
    if i>10:
        filter_z_score.append(i)
print(filter_z_score)

[10.083822881245718, 18.309067251750708, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 13.951522052424636, 10.083822881245718, 45.557013123834146, 27.757884467031786, 10.083822881245718, 10.083822881245718, 27.757884467031786, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 12.927014295682822, 27.757884467031786, 26.073647910679014, 31.532825560540285, 46.49501615645636, 35.75175475615663, 45.557013123834146, 27.897542696333314, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 10.083822881245718, 16.55187490397177, 37.20357500553745, 22.11944401538049, 10.083822881245718, 10.083822881245718, 10.083822881245718, 23.579602069495735, 10.083822881245718, 35.75279698174843, 13.951522052424636, 10.083822881245718, 10.083822881245718, 16.91

In [38]:
len(filter_z_score)

62

In [39]:
answer_four = spainEnglishAll[spainEnglishAll["z_score_retweet"]>10.00]

In [40]:
answer_four.shape

(62, 11)

In [41]:
answer_four

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
859,859,1461469510586839047,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 23:00:58+00:00,1036271517699587968,en,9909,0,0,49545,10
2252,2252,1461451876000489474,RT @MesutOzil1088: I grew up as a @Fenerbahce ...,2021-11-18 21:50:54+00:00,1245113611263057920,en,17801,0,0,89005,18
3009,3009,1461442074184720387,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 21:11:57+00:00,955569674737127424,en,9909,0,0,49545,10
4527,4527,1461421081105948679,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:48:32+00:00,1097937228607700992,en,9909,0,0,49545,10
4585,4585,1461420287510159373,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:45:23+00:00,1371813880343711488,en,9909,0,0,49545,10
5082,5082,1461415007846117376,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:24:24+00:00,2949011187,en,9909,0,0,49545,10
5403,5403,1461413368959492108,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:17:53+00:00,16095339,en,9909,0,0,49545,10
5484,5484,1461412419377537025,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:14:07+00:00,2385072439,en,9909,0,0,49545,10
5530,5530,1461411825971515395,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:11:45+00:00,984144046833721216,en,9909,0,0,49545,10
6012,6012,1461405606540103682,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 18:47:02+00:00,2766567177,en,9909,0,0,49545,10


## Question 5. Find the top 5 amplifiers (15 points)

### <font color='green'>We consider amplifiers those twitter users who are involved in highly amplified conversations.</font>

In [42]:
spainEnglishAll.groupby(["tweet.author_id"]).mean("amplification_volume").sort_values("amplification_volume", ascending=False).head(5)

,Unnamed: 0,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
tweet.author_id,,,,,,
1168957082,8820,43945,0,0,219725,46
1064006291087970432,39357,35930,0,0,179650,37
1101831422631804800,23656,35161,0,0,175805,36
1290910527393210368,23426,31863,0,0,159315,33
1322715465198821376,44456,30490,0,0,152450,32


### <font color='green'>Answer: For this question, we need to look average number of amplification of every tweets for per tweet.author_id. If the number is high, these accounts only retweet, like or reply in high amplified conversations in this given dataset.</font>

In [43]:
grouped_tweet_text.head(5)

,Unnamed: 0,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume
tweet.text,,,,,,
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography \n#NCT #NCT127 #JAEHYUN\n#Spain #Sevilla https://t.co/pzRbU9G82o,23467,1290910527393210368,44845,0,0,224225
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography\n#NCT #NCT127\n#JAEHYUN #MARK \n#Spain #Sevilla https://t.co/jPNpBlVUcJ,16238,550915711900380928,43945,0,0,219725
RT @NCTsmtown_127: Spain here we goooooo\n\n#NCT #NCT127 #MARK\n#NCT127inSpain https://t.co/3kuRW5nG1f,39357,1064006291087970432,35930,0,0,179650
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#Johntography\n#NCT #NCT127 #DOYOUNG \n#Spain #Sevilla https://t.co/XFbjF6fXt2,36979,1290852263092957184,34538,0,0,172688
RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain\n\n#NCT #NCT127 \n#Spain #Sevilla https://t.co/2xGKANRyR2,45333,1382234880465989632,32878,0,0,164390


In [44]:
spainEnglishAll[spainEnglishAll["tweet.author_id"]==1168957082]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
8820,8820,1461371211343020033,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 16:30:22+00:00,1168957082,en,43945,0,0,219725,46


In [45]:
spainEnglishAll[spainEnglishAll["tweet.author_id"]==1064006291087970432]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
39357,39357,1461649200882405377,RT @NCTsmtown_127: Spain here we goooooo\n\n#N...,2021-11-19 10:55:00+00:00,1064006291087970432,en,35930,0,0,179650,37


In [46]:
spainEnglishAll[spainEnglishAll["tweet.author_id"]==1101831422631804800]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
23655,23655,1461157789397438464,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 02:22:18+00:00,1101831422631804800,en,34537,0,0,172685,36
23656,23656,1461157780664901633,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 02:22:16+00:00,1101831422631804800,en,43945,0,0,219725,46
23658,23658,1461157760607739909,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 02:22:11+00:00,1101831422631804800,en,27001,0,0,135005,28


In [47]:
spainEnglishAll[spainEnglishAll["tweet.author_id"]==1290910527393210368]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
23311,23311,1461167495809617921,RT @NCTsmtown_127: #Johnny's Photo Diary:Spain...,2021-11-18 03:00:52+00:00,1290910527393210368,en,26867,0,0,134335,28
23461,23461,1461162923057827840,RT @NCTsmtown_127: Amazing time in SPAIN🇪🇸\nGr...,2021-11-18 02:42:42+00:00,1290910527393210368,en,25251,0,0,126255,26
23465,23465,1461162702307426312,RT @NCTsmtown_127: #Johnny 's Photo Diary:Spai...,2021-11-18 02:41:49+00:00,1290910527393210368,en,30489,0,0,152445,32
23467,23467,1461162679809155076,RT @NCTsmtown_127: #Johnny’s Photo Diary:Spain...,2021-11-18 02:41:44+00:00,1290910527393210368,en,44845,0,0,224225,46


In [48]:
spainEnglishAll[spainEnglishAll["tweet.author_id"]==1322715465198821376]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet
44456,44456,1461565440879206401,RT @NCTsmtown_127: #Johnny 's Photo Diary:Spai...,2021-11-19 05:22:10+00:00,1322715465198821376,en,30490,0,0,152450,32


###  <font color='green'>As we can see, tweet.author_id numbers 1290910527393210368 and 1101831422631804800 have joined highest amplified conversations, but they also have joined less amplified conversations too. Thus, their average amplification_volume got decrease.</font>

## Question 6. Extract, from the text colum, information on who created the conversation in the first place (15 points)
### Tip: retweeted conversations start with the pattern RT@
### For illustration purposes, the text "RT @20splentyforus: France, like UK should set a national 30km/h or 20mph urban default limit." was originally created by @20splentyforus
### Tip2. consider applying a regex pattern to the DataFrame using apply method

In [49]:
spainEnglishAll["tweet.text"].dtype

dtype('O')

In [50]:
spainEnglishAll["public_metrics.retweet_count"].dtype

dtype('float64')

In [51]:
tweet_text_str = spainEnglishAll["tweet.text"].astype(str)

In [52]:
original_accountt = tweet_text_str.str.extract(r"(^RT.@[a-zA-Z|\d]*.[a-zA-Z|\d]*)", expand=False)

In [53]:
original_accountt

0                RT @ESPNFC:
1            RT @MrGeepGeep:
2           RT @AnittaCrave:
3                        NaN
4            RT @Foreign_Cat
                ...         
47545                    NaN
47546    RT @ancientorigins:
47547                    NaN
47548      RT @THEMAXHILTON:
47549                    NaN
Name: tweet.text, Length: 47550, dtype: object

In [54]:
spainEnglishAll["original_account"] = original_accountt

In [55]:
spainEnglishAll.loc["34123":"34234"]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,amplification_volume,z_score_retweet,original_account
34123,34123,1461719597602672643,RT @FootballlForAll: 🚨⚽️ | Fifa have released ...,2021-11-19 15:34:44+00:00,1393559650465943552,en,93,0,0,465,-0,RT @FootballlForAll:
34124,34124,1461719594046038017,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-19 15:34:43+00:00,1442550952611889152,en,1568,0,0,7840,1,RT @ESPNFC:
34125,34125,1461719585166667776,LOVE SummerTime TV. Ocean City Christmas light...,2021-11-19 15:34:41+00:00,364789232,en,0,1,0,4,-0,NaN
34126,34126,1461719583862177792,RT @JusticeTourNews: .@justinbieber’s “Justice...,2021-11-19 15:34:40+00:00,1137080751135100800,en,310,0,0,1550,0,RT @JusticeTourNews:
34127,34127,1461719564551528451,Boris 'CLEARED' over refusal to declare Spain ...,2021-11-19 15:34:36+00:00,1420660507,en,0,0,0,0,-0,NaN
34128,34128,1461719560449626125,RT @CanYamanMedia: 🇪🇸🗞| #CanYaman in Spain: th...,2021-11-19 15:34:35+00:00,1211884597924909056,en,59,0,0,295,-0,RT @CanYamanMedia:
34129,34129,1461719540786733064,@Abelsinho @FabrizioRomano @Ramon_AlvarezMM I’...,2021-11-19 15:34:30+00:00,875877882,en,0,1,0,4,-0,NaN
34130,34130,1461719508901666822,⬛️⬜️‼️ #Spain's police considers that protesti...,2021-11-19 15:34:23+00:00,732700920,en,17,0,12,109,-0,NaN
34131,34131,1461719502257889290,RT @sat_purusha: #FollowBackFriday... It's col...,2021-11-19 15:34:21+00:00,124296126,en,47,0,0,235,-0,RT @sat_purusha
34132,34132,1461719483471573003,🇪🇸🗞| #CanYaman in Spain: this will be the retu...,2021-11-19 15:34:16+00:00,1284939145522089984,en,59,1,190,679,-0,NaN


### <font color='green'>Answer: original_account column has the account names who have started the conversations.</font>

## Question 7. Please briefly reflect on how would you try to mitigate misinformation on social media (2 paragraphs maximum). (10 points)

####  We are living in communication era, so number of conversations have dramatically increased. This improvement provides great individual power that ability to increase people’ voice, also birth to potential risks such as suitable environment for spreading misinformation. I believe that there are two aspects of solution in this problem, which are external aspect and internal aspect. The solution can be only reached if these two sides work together. External aspect includes regulators such as states and international institutes that has authority to declare rule for mitigating misinformation. External factors can create standard misinformation organizations in a global and regional scales, so they can do necessary research and determine scientific steps to prevent spreading of misinformation by auditing social media companies.  

#### I believe one solution cannot be enough for this problem, so private companies have to try to prevent this problem too. My second potential solution that can help other solutions, is that companies can grade the posts rather than profiles. To illustrate, if someone has a claim about the history, he or she needs to show source to make his or her post validation sign. It does not matter where the person gets source in order to get validation sign. Person just need to show the source and then directly gets validation sign. Even the source is wrong at least people can check the trust level of the source easier than post that has 240 characters from unknown person. 